In [1]:
%tensorflow_version 1.x
from skimage import measure, transform
import matplotlib.pyplot as plt
from keras import backend as K
import nibabel as nib
import numpy as np
import glob
import math
import time

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
def fft_lr(img, factor):
    x = int(img.shape[1]//(factor*2))
    imgfft = np.fft.fftn(img)
    imgfft = np.fft.fftshift(imgfft)
    y_center = imgfft.shape[1] // 2
    z_center = imgfft.shape[2] // 2
    imgfft = imgfft[:, y_center-x : y_center+x, z_center-x : z_center+x]
    imgifft = np.fft.ifftn(imgfft)
    lr_img = abs(imgifft)

    lr_img = transform.resize(lr_img, img.shape)
    lr_img /= lr_img.max()
    lr_img *= 255
    return lr_img

In [3]:
def PSNR(y_true, y_pred):
    max_pixel = 255.0
    return 10.0 * (1.0 / math.log(10)) * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))

In [4]:
def psnr(y_true, y_pred):
    # y_true = y_true.astype(np.float64) / 255.
    # y_pred = y_pred.astype(np.float64) / 255.
    mse = np.mean( (y_true - y_pred) ** 2 )
    if mse == 0:
      return 100
    return 20 * math.log10(255 / math.sqrt(mse))

In [ ]:
from keras.models import load_model
m2d = load_model('/content/drive/My Drive/Colab Notebooks/final/model_2d_fft.h5')
m3d = load_model('/content/drive/My Drive/Colab Notebooks/final/model_3d_915_32x32.h5')
m3df = load_model('/content/drive/My Drive/Colab Notebooks/final/fsrcnn_56_16_relu_2.h5', custom_objects={'PSNR': PSNR})

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\gputest\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-623ae21f8c21>", line 2, in <module>
    m2d = load_model('/content/drive/My Drive/Colab Notebooks/final/model_2d_fft.h5')
  File "C:\Users\user\anaconda3\envs\gputest\lib\site-packages\keras\models.py", line 237, in load_model
    with h5py.File(filepath, mode='r') as f:
  File "C:\Users\user\anaconda3\envs\gputest\lib\site-packages\h5py\_hl\files.py", line 445, in __init__
    swmr=swmr)
  File "C:\Users\user\anaconda3\envs\gputest\lib\site-packages\h5py\_hl\files.py", line 195, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py\h5f.pyx", line 96, in h5py.h5f.open
FileNotFoundError: [Errno 2] Una

In [0]:
files = (glob.glob("/content/drive/My Drive/scanss/*.nii.gz"))

In [44]:
#for 2dsrcnn
timez = 0
p = 35
s = 0.95
maxp = 0
minp = 100
maxs = 0
mins = 100
for image in range (len(files)):
    start_time = time.time()
    img = nib.load(files[image])
    img_data = img.get_fdata()
    img_data = img_data/(np.amax(img_data))
    img_data = img_data * 255
    hr = np.array(img_data, dtype='uint8')

    lr = fft_lr(hr, 2)
    lr = np.array(lr, dtype='uint8')
    for x in range(hr.shape[0]):
        hr1 = hr[x,:,:]
        lr1 = lr[x,:,:]
        lr1_1 = np.reshape(lr1, (1, hr1.shape[0], hr1.shape[1], 1))

        sr1 = m2d.predict(lr1_1)
        sr1 = np.reshape(sr1,hr1.shape)
        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    for y in range(hr.shape[1]):
        hr1 = hr[:,y,:]
        lr1 = lr[:,y,:]
        lr1_1 = np.reshape(lr1, (1, hr1.shape[0], hr1.shape[1], 1))

        sr1 = m2d.predict(lr1_1)
        sr1 = np.reshape(sr1,hr1.shape)
        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    for z in range(hr.shape[2]):
        hr1 = hr[:,:,z]
        lr1 = lr[:,:,z]
        lr1_1 = np.reshape(lr1, (1, hr1.shape[0], hr1.shape[1], 1))

        sr1 = m2d.predict(lr1_1)
        sr1 = np.reshape(sr1,hr1.shape)
        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    timez += (time.time() - start_time)

print()
print("Max PSNR(2DSRCNN):", maxp)
print("Average PSNR(2DSRCNN):", p)
print("Min PSNR(2DSRCNN):", minp)
print()
print("Max SSIM(2DSRCNN):", maxs)
print("Average SSIM(2DSRCNN):", s)
print("Min SSIM(2DSRCNN):", mins)
print()
print("Average time(second) for each scan:", timez/len(files))
print()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/skimage/measure/_structural_similarity.py:17: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  **kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.



Max PSNR(2DSRCNN): 37.155619744747895
Average PSNR(2DSRCNN): 32.49401813642935
Min PSNR(2DSRCNN): 24.97284103292985

Max SSIM(2DSRCNN): 0.9844260463489186
Average SSIM(2DSRCNN): 0.8608954532266173
Min SSIM(2DSRCNN): 0.8102512056702726

Average time(second) for each scan: 3.1840524514516195



In [39]:
#for 3dsrcnn
timez = 0
p = 35
s = 0.95
maxp = 0
minp = 100
maxs = 0
mins = 100
for image in range (len(files)):
    start_time = time.time()
    img = nib.load(files[image])
    img_data = img.get_fdata()
    img_data = img_data/(np.amax(img_data))
    img_data = img_data * 255
    hr = np.array(img_data, dtype='uint8')

    test_lr = fft_lr(hr, 2.5)
    test_lr = np.array(test_lr, dtype='uint8')
    test_sr = np.ndarray((256, 320, 320), dtype='uint8')

    for i in range(0, 256, 64):
        for j in range(0, 320, 64):
            for k in range(0, 320, 64):
                lrx = test_lr[i:i+64, j:j+64, k:k+64]
                lrx = np.reshape(lrx, (1, lrx.shape[0], lrx.shape[1], lrx.shape[2], 1))
                srx = m3d.predict(lrx)
                srx = np.reshape(srx, (64,64,64))
                test_sr[i:i+64, j:j+64, k:k+64] = srx
    timez += time.time() - start_time

    for x in range(hr.shape[0]):
        hr1 = hr[x,:,:]
        sr1 = test_sr[x,:,:]

        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    for y in range(hr.shape[1]):
        hr1 = hr[:,y,:]
        sr1 = test_sr[:,y,:]

        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    for z in range(hr.shape[2]):
        hr1 = hr[:,:,z]
        sr1 = test_sr[:,:,z]

        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

print()
print("Max PSNR(3DSRCNN):", maxp)
print("Average PSNR(3DSRCNN):", p)
print("Min PSNR(3DSRCNN):", minp)
print()
print("Max SSIM(3DSRCNN):", maxs)
print("Average SSIM(3DSRCNN):", s)
print("Min SSIM(3DSRCNN):", mins)
print()
print("Average time(second) for each scan:", timez/len(files))
print()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.



Max PSNR(3DSRCNN): 41.22989450492577
Average PSNR(3DSRCNN): 35.253762055833256
Min PSNR(3DSRCNN): 29.290997790650188

Max SSIM(3DSRCNN): 0.976620152412472
Average SSIM(3DSRCNN): 0.904876076269809
Min SSIM(3DSRCNN): 0.7497611097528034

Average time(second) for each scan: 11.454299628734589



In [40]:
#for 3dfsrcnn
timez = 0
p = 35
s= 0.95
maxp = 0
minp = 100
maxs = 0
mins = 100
for image in range (len(files)):
    start_time = time.time()
    img = nib.load(files[image])
    img_data = img.get_fdata()
    img_data = img_data/(np.amax(img_data))
    img_data = img_data * 255
    hr = np.array(img_data, dtype='uint8')

    lr = hr[::2, ::2, ::2]
    sr = np.zeros((256, 320, 320), dtype='uint8')
    x = 0
    x1 = 0
    y = 0
    y1 = 0
    z = 0
    z1 = 0

    for i in range(4):
        y = 0
        y1 = 0
        for j in range(5):
            z = 0
            z1 = 0
            for k in range(5):
                lr0 = np.zeros((1, 32, 32, 32), dtype='uint8')
                lr0[0] = lr[x:x+32, y:y+32, z:z+32]
                lr0.shape = lr0.shape + (1,)
                sr0 = m3df.predict(lr0)

                sr0 = np.reshape(sr0, (1,64,64,64))
                sr[x1:x1+64, y1:y1+64, z1:z1+64] = sr0[0]

                z = z + 32
                z1 = z1 + 64
            y = y + 32
            y1 = y1 + 64
        x = x + 32
        x1 = x1 + 64

    timez += time.time() - start_time

    for x in range(hr.shape[0]):
        hr1 = hr[x,:,:]
        sr1 = sr[x,:,:]

        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    for y in range(hr.shape[1]):
        hr1 = hr[:,y,:]
        sr1 = sr[:,y,:]

        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

    for z in range(hr.shape[2]):
        hr1 = hr[:,:,z]
        sr1 = sr[:,:,z]

        p = (p + psnr(hr1, sr1))/2
        s0, d = measure.compare_ssim(hr1, sr1, full=True)
        s = (s + s0)/2
        if p > maxp:
            maxp = p
        if p < minp:
            minp = p
        if s > maxs:
            maxs = s
        if s < mins:
            mins = s

print()
print("Max PSNR(3DFSRCNN):", maxp)
print("Average PSNR(3DFSRCNN):", pg)
print("Min PSNR(3DFSRCNN):", minp)
print()
print("Max SSIM(3DFSRCNN):", maxs)
print("Average SSIM(3DFSRCNN):", s)
print("Min SSIM(3DFSRCNN):", mins)
print()
print("Average time(second) for each scan:", timez/len(files))
print()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.



Max PSNR(3DFSRCNN): 48.66321216684466
Average PSNR(3DFSRCNN): 38.756011903383595
Min PSNR(3DFSRCNN): 34.47256423180401

Max SSIM(3DFSRCNN): 0.9853241307472062
Average SSIM(3DFSRCNN): 0.9812111155616676
Min SSIM(3DFSRCNN): 0.8471021333569505

Average time(second) for each scan: 1.9810153047243755

